In [ ]:
#Jadi Tambah Jumlah Tweet deng Form pickle fix
import tweepy
import time
import re
import numpy as np
import pandas as pd
from keras.models import load_model
from flask import Flask, render_template, request, redirect, url_for
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from numpy import array
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
import string
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
import plotly.express as px
import datetime
from IPython.display import clear_output
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pickle


pd.set_option('display.max_colwidth', 1000)

api_key = ""
api_secret_key = ""
access_token = ""
access_token_secret = ""

authentication = tweepy.OAuthHandler(api_key, api_secret_key)
authentication.set_access_token(access_token, access_token_secret)
api = tweepy.API(authentication, wait_on_rate_limit=True)

def get_related_tweets(text_query):
    tweets_list = []
    count = 250
    try:
        #parameter search keyword
        for tweet in tweepy.Cursor(api.search, q=text_query, lang='id',tweet_mode = 'extended', trucated='false').items(100):
        #cleansing retweet
                tweettext = str(tweet.full_text.encode('ascii',errors='ignore'))
                if tweettext.startswith("rt @") == False and hasattr(tweet, 'retweeted_status') == False:
                    tweets_list.append({'created_at': tweet.created_at,
                                               'tweet_id': tweet.id,
                                               'tweet_text': tweet.full_text})
                    
                    tweetx = pd.DataFrame.from_dict(tweets_list)
                    tweetx2 = pd.DataFrame.from_dict(tweets_list)
                    tweetxy = tweetx['tweet_text']
                    tweettanggall = tweetx['created_at'].astype(str)
                    tweettanggal = str(tweetx['created_at'])
                    
                    #cleansing
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'[@#][A-Za-z0-9_]+', '')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace('https://', '')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace('t.co/[a-zA-z0-9]+', '')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.lstrip(':b')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'\\x[a-z0-9]+', '')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'\\n', ' ')
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'[^\w\s]',' ', re.UNICODE)
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'[-._]',' ', re.UNICODE)
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.replace(r'[0-9]+', '')
                    #case folding
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].str.lower()
                    #token
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].apply(word_tokenize)
                    #stopword
                    factory = StopWordRemoverFactory()
                    stopwords = factory.get_stop_words()
                    stopwords.extend(['g','ae', 'y', 'a', 'i','ye', 'ya', 'da', 'yg','hm','hmm' , 'rt', 
                                      'amp', 'bilang', 'krn', 'ny','nya', 'd', 'tuh', 'utk', 'ya', 'n', 
                                      'hehe', 'loh', 'rt', '&amp','amp','yah'])
                    stopwords = set(stopwords) - {'tidak','ada', 'belum', 'mengapa', 'kenapa', 'boleh' , 
                                                  'nggak', 'tolong', 'ingin', 
                                                  'bisa', 'nanti','dahulu', 'bilang', 'kecuali', 'tentang',
                                                  'tetapi', 'dapat',  'daripada'}
                    tweetx2['tweet_text'] =  tweetx2['tweet_text'].apply(lambda x: [item for item in x if 
                                                                                    item not in stopwords])
                    tweetx2.to_csv('data2.csv')
                    datacsv = pd.read_csv('data2.csv')
                    
        return tweetx, tweettanggal, tweetxy ,tweettanggall, datacsv, text_query;
    except BaseException as e:
        print('failed on_status,', str(e))
        time.sleep(3)

        
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "GET":
        return render_template("htmlforscrapping.html")
    elif request.method == "POST":
        user = request.form['name']
        return redirect(url_for('success', name=user))
    
@app.route('/success/<name>', methods=["GET", "POST"])
def success(name):
    if request.method == "GET":
        global model,graph
        graph = tf.get_default_graph()
        model = load_model('weights3.hdf5')
      
        #tokenizer, index, padding
        tweetx, tweetxy, tweettanggal, tweettanggall, datacsv, text_query = get_related_tweets(name)
        que = text_query
        
        with open('tokenizer1.pcl', 'rb') as handle:
            tokenizer2 = pickle.load(handle)
        #print(que)
        x_test = tokenizer2.texts_to_sequences(datacsv['tweet_text'])
        x_test = pad_sequences(x_test, maxlen=47)
        
        hasil = model.predict(x_test)
        prediksi2 = np.argmax(hasil, axis=1)
        hasilprediksi = []
        for i in range(len(prediksi2)):
            if prediksi2[i] == 2:
                hasilprediksi.append("Positif")
            elif prediksi2[i] == 1:
                hasilprediksi.append("Netral")
            else:
                hasilprediksi.append("Negatif")  
        
        #pie chart
        Positif, Netral, Negatif = 0, 0, 0
        for i, prediksi in enumerate(hasilprediksi):
            if prediksi=="Positif":
                Positif += 1
            elif prediksi=="Netral":
                Netral += 1
            else:
                Negatif += 1   
        
        labels=['[Positif]','[Netral]', '[Negatif]']
        sizes=[Positif, Netral, Negatif]   
        graph_values = [{
                    'labels': labels,
                    'values': sizes,
                    'type': 'pie',
                    'insidetextfont': {'color': '#FFFFFF',
                                        'size': '14',
                    },
                    'textfont': {'color': '#FFFFFF',
                                        'size': '14',
                    },
                    }]

        layout = {
                'title': '<b>Rekapitulasi Sentimen</b>',

                }
        
        #for keyword in each tweet(database csv)
        hasiltweet = tweetx['tweet_text']
        hasiltweets = hasiltweet.tolist()
        q_text = []
        for z in range (len(hasiltweets)):
            q_text.append(que)
        
        headings = ("Tanggal", "Tweet", "Sentimen")
        data = list(zip(tweettanggall, hasiltweets, hasilprediksi))
    
        dict1 = {"Keyword": q_text, "Tanggal": tweettanggall, "Tweet": hasiltweets, "Sentimen": 
                 hasilprediksi}
        df = pd.DataFrame(dict1, columns= ["Keyword", "Tanggal", "Tweet", "Sentimen"])
        df.to_csv('tes3.csv', mode='a', header=False)
    
        #Bekeng line chart
        df = pd.read_csv('tes3.csv', header=None, 
                 names=["Keyword", "Tanggal", "Tweet", "Sentimen"])
        df['Tanggal'] = pd.to_datetime(df['Tanggal'], errors='coerce')


        mask = df['Keyword']==que
        result = df[mask].groupby([pd.Grouper(key='Tanggal', freq='60s'), 'Sentimen']).count().unstack(fill_value=0).stack().reset_index()
        result = result.rename(columns={"Keyword": "Jumlah Sentimen", "Tanggal":"Waktu"})
        time_series = result["Waktu"][result['Sentimen']==0].reset_index(drop=True)


        fig = px.line(result, x='Waktu',                      \
              y="Jumlah Sentimen",
              color="Sentimen")      
    
        fig = fig.update_xaxes(rangeslider_visible=True)
        fig.update_layout(width=750, height=500)
        graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    
        return render_template("fortabel.html", headings=headings, data=data, 
                               que=que, graph_values=graph_values, layout=layout, 
                               graphJSON=graphJSON)
    
    elif request.method == "POST":
        user = request.form['nama']
        return redirect(url_for('success', name=user))
    
if __name__ == "__main__":
    app.run(debug=False)  